# Project Yelp API

## <font color=darkblue>About</font>
<font color=darkblue> *This notebook was created to collect yelp review data for another fast food restuarnt, KFC. It uses the official Yelp API and makes calls to the Business Search and Business Review Endpoints. This additional data was used to try out our model on the "mod4_project" notebook to classify negative yelp reviews for KFC.* </font>

### 1. Import Libraries and API Key

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from yelp.client import Client
import config
import itertools

In [ ]:
#use YELP API key here
MY_API_KEY = config.api_key

### 2. Business Search Endpoint

In [ ]:
#function that returns number of business (1000 max) based on search criteria
def fast_food(restaurant, locale, int):
    '''
    Makes Yelp API calls using the Business Search Endpoint.
    
    First two inputs should be strings (search term and location) and 
    the third should be an integer (search limit)
    
    Example) fast_food('KFC', 'New York, NY', 50)
    '''
    
    MY_API_KEY = config.api_key
    term = restaurant
    location = locale
    SEARCH_LIMIT = int

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(MY_API_KEY),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT
            }
    response = requests.get(url, headers=headers, params=url_params)
    print(response)
    print(type(response.text))
    return response.text

In [ ]:
#Use function for KFC's located in specific area
kfc_lv = fast_food('KFC', 'Las Vegas, NV', 50)
kfc_lv = json.loads(kfc_lv)

In [ ]:
kfc_lv_df = pd.DataFrame.from_dict(kfc_lv['businesses'])
kfc_lv_df

In [ ]:
kfc_la = fast_food('Burger King', 'Los Angeles, CA', 50)
kfc_la = json.loads(kfc_la)

In [ ]:
kfc_la_df = pd.DataFrame.from_dict(kfc_la['businesses'])
kfc_la_df

In [ ]:
kfc_atl = fast_food('KFC', 'Atlanta, GA', 50)
kfc_atl = json.loads(kfc_atl)

In [ ]:
kfc_atl_df = pd.DataFrame.from_dict(kfc_atl['businesses'])
kfc_atl_df

In [ ]:
kfc_chi = fast_food('KFC', 'Chicago, IL', 50)
kfc_chi = json.loads(kfc_chi)

In [ ]:
kfc_chi_df = pd.DataFrame.from_dict(kfc_chi['businesses'])
kfc_chi_df

In [ ]:
kfc_cl = fast_food('KFC', 'Cleveland, OH', 50)
kfc_cl = json.loads(kfc_cl)

In [ ]:
kfc_cl_df = pd.DataFrame.from_dict(kfc_cl['businesses'])
kfc_cl_df

In [ ]:
kfc_dal = fast_food('KFC', 'Dallas, TX', 50)
kfc_dal = json.loads(kfc_dal)

In [ ]:
kfc_dal_df = pd.DataFrame.from_dict(kfc_dal['businesses'])
kfc_dal_df

In [ ]:
kfc_ho = fast_food('KFC', 'Houston, TX', 50)
kfc_ho = json.loads(kfc_ho)

In [ ]:
kfc_ho_df = pd.DataFrame.from_dict(kfc_ho['businesses'])
kfc_ho_df

In [ ]:
kfc_ny = fast_food('KFC', 'New York, NY', 50)
kfc_ny = json.loads(kfc_ny)

In [ ]:
kfc_ny_df = pd.DataFrame.from_dict(kfc_ny['businesses'])
kfc_ny_df

In [ ]:
kfc_por = fast_food('KFC', 'Portland, OR', 50)
kfc_por = json.loads(kfc_por)

In [ ]:
kfc_por_df = pd.DataFrame.from_dict(kfc_por['businesses'])
kfc_por_df

In [ ]:
#append all KFC locations into one df
kfc_biz = kfc_atl_df.append([kfc_chi_df, kfc_cl_df, kfc_dal_df, kfc_ho_df, kfc_la_df, kfc_lv_df, kfc_ny_df, kfc_por_df]).reset_index()

In [ ]:
kfc_biz

In [ ]:
indexNames = kfc_biz[kfc_biz['name']!= 'KFC'].index

In [ ]:
#drop rows that don't have KFC as the name
kfc_biz.drop(indexNames, inplace = True)

In [ ]:
#get business ids into a list to use for calling the Business Review Endpoint
kfc_id = kfc_biz['id'].tolist()

### 3. Business Reviews Endpoint

In [ ]:
#uses all the business id's to get the 3 reviews and other info from each KFC into a list

kfc_reviews = []
for id in kfc_id: 
    url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(id)
    headers = {
        'Authorization': 'Bearer {}'.format(MY_API_KEY),
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    kfc_reviews.append(data)
print(kfc_reviews)

In [ ]:
kfc_df = pd.DataFrame(kfc_reviews)

In [ ]:
kfc_df

In [ ]:
#check for any null values
kfc_df[kfc_df.reviews.isnull()]

In [ ]:
#remove null rows
kfc_df = kfc_df[kfc_df['reviews'].notna()]

In [ ]:
#convert reviews column into it's own list to make a separate dataframe
kfc_rev = kfc_df['reviews'].tolist()

In [ ]:
#concatenate the list of lists into one full list
kfc_review = list(itertools.chain.from_iterable(kfc_rev))

In [ ]:
#convert into a new df, containing review id, url, review text, rating, etc
kfc_rev_df = pd.DataFrame(kfc_review)

In [ ]:
#remove any reviews with a star rating less than 3 since we only want to run the model on negative reviews
kfc_lo_rev = kfc_rev_df.loc[kfc_rev_df['rating']< 3]

### 4. Merge Business Search and Business Review Dataframes

In [ ]:
#split the url column so we can use to merge with kfc_biz
kfc_lo_rev['url'] = kfc_lo_rev['url'].str.split('?').str[0]

In [ ]:
kfc_biz['url'] = kfc_biz['url'].str.split('?').str[0]

In [ ]:
kfc = kfc_lo_rev.merge(kfc_biz, on = 'url', how = 'left')


In [ ]:
kfc.head(21)